In [ ]:
!pip install captum
!pip install -U bitsandbytes

In [1]:
import gc
import torch
import pandas as pd
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from captum.attr import LLMGradientAttribution, LayerIntegratedGradients, TextTokenInput

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")

In [2]:
login(token = HF_TOKEN)

In [3]:
df = pd.read_csv("/kaggle/input/redteaminterp/cm_cmp_concatenated.csv")
cm = df['text'][:362]
cmp = df['text'][362:]

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using GPU


In [ ]:
del model

In [5]:
torch.cuda.empty_cache()
gc.collect()

90

In [6]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
tokenizer.pad_token = tokenizer.eos_token
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True
)

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    device_map="auto",
    quantization_config=bnb_config
)

2025-04-28 18:41:45.526194: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745865705.548896     190 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745865705.555746     190 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
lig = LayerIntegratedGradients(model, layer=model.model.embed_tokens)
llm_attr = LLMGradientAttribution(lig, tokenizer)

In [8]:
cm_prompt = cm.iloc[0]
cmp_prompt = cmp.iloc[0]
safe_response = "I cannot provide information or guidance on illegal or harmful activities, including medical experiments without consent. Can I help you with something else?"

In [9]:
skip_tokens = [1]
inp = TextTokenInput(
    cm_prompt,
    tokenizer,
    skip_tokens=skip_tokens,
)

attr_res = llm_attr.attribute(inp, target=safe_response, skip_tokens=skip_tokens)

attr_res.plot_seq_attr(show=True)

/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 1 has a total capacity of 14.74 GiB of which 208.12 MiB is free. Process 22410 has 14.54 GiB memory in use. Of the allocated memory 13.84 GiB is allocated by PyTorch, and 584.00 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)